In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("../dataset/Transaction.csv")
rfm = pd.read_excel("../dataset/rfm_mapping.xlsx")

In [3]:
df.head()

,id,UserId,TransactionId,TransactionTime,ItemCode,ItemDescription,NumberOfItemsPurchased,CostPerItem,Country
0,0,278166,6355745,2019-02-02 12:50:00,465549,FAMILY ALBUM WHITE PICTURE FRAME,6,11.73,United Kingdom
1,1,337701,6283376,2018-12-26 09:06:00,482370,LONDON BUS COFFEE MUG,3,3.52,United Kingdom
2,2,267099,6385599,2019-02-15 09:45:00,490728,SET 12 COLOUR PENCILS DOLLY GIRL,72,0.90,France
3,3,380478,6044973,2018-06-22 07:14:00,459186,UNION JACK FLAG LUGGAGE TAG,3,1.73,United Kingdom
4,4,285957,6307136,2019-01-11 09:50:00,1787247,CUT GLASS T-LIGHT HOLDER OCTAGON,12,3.52,United Kingdom


In [4]:
rfm.head()

,r_score,f_score,m_score,RFM_score,Segments
0,3,1,4,314,Big spender
1,3,1,5,315,Big spender
2,3,2,4,324,Big spender
3,3,2,5,325,Big spender
4,3,3,4,334,Big spender


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380405 entries, 0 to 380404
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      380405 non-null  int64  
 1   UserId                  380405 non-null  int64  
 2   TransactionId           380405 non-null  int64  
 3   TransactionTime         380405 non-null  object 
 4   ItemCode                380405 non-null  int64  
 5   ItemDescription         380405 non-null  object 
 6   NumberOfItemsPurchased  380405 non-null  int64  
 7   CostPerItem             380405 non-null  float64
 8   Country                 380405 non-null  object 
dtypes: float64(1), int64(5), object(3)
memory usage: 26.1+ MB


In [6]:
df["Amount"] = df["CostPerItem"] * df ["NumberOfItemsPurchased"]

In [7]:
monetary_df = df.groupby("UserId")["Amount"].sum().reset_index()
monetary_df.head()

,UserId,Amount
0,259266,320608.80
1,259287,17565.78
2,259308,6954.48
3,259329,7211.94
4,259350,1387.20


In [8]:
frequency_df = df.groupby("UserId")["TransactionId"].nunique().reset_index()
frequency_df.head()

,UserId,TransactionId
0,259266,1
1,259287,7
2,259308,4
3,259329,1
4,259350,1


In [9]:
df["TransactionTime"] = pd.to_datetime(df["TransactionTime"])

In [10]:
today = pd.to_datetime("today").normalize()
reference_date = today
recency_df = df.groupby("UserId")["TransactionTime"].max().reset_index()
recency_df["Recency"] = (reference_date - recency_df["TransactionTime"]).dt.days
recency_df.head()

,UserId,TransactionTime,Recency
0,259266,2018-04-01 06:01:00,2656
1,259287,2019-02-18 11:52:00,2333
2,259308,2018-12-07 09:13:00,2406
3,259329,2019-02-02 05:51:00,2349
4,259350,2018-04-16 12:01:00,2641


In [11]:
recency_df = recency_df.drop("TransactionTime",axis=1)
recency_df.head()

,UserId,Recency
0,259266,2656
1,259287,2333
2,259308,2406
3,259329,2349
4,259350,2641


In [12]:
recency_df = recency_df.merge(frequency_df,on = "UserId")
rfmdf = recency_df.merge(monetary_df,on = "UserId")
rfmdf.head()

,UserId,Recency,TransactionId,Amount
0,259266,2656,1,320608.80
1,259287,2333,7,17565.78
2,259308,2406,4,6954.48
3,259329,2349,1,7211.94
4,259350,2641,1,1387.20


In [13]:
rfmdf["Rvalue"] = pd.qcut(rfmdf["Recency"], 5, labels=[5, 4, 3, 2, 1])
rfmdf["Fvalue"] = pd.qcut(rfmdf["TransactionId"].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
rfmdf["Mvalue"] = pd.qcut(rfmdf["Amount"], 5, labels=[1, 2, 3, 4, 5])

In [14]:
rfmdf["RFM_score"] = (rfmdf["Rvalue"].astype(str) + rfmdf["Fvalue"].astype(str) + rfmdf["Mvalue"].astype(str))

In [15]:
rfmdf.head()

,UserId,Recency,TransactionId,Amount,Rvalue,Fvalue,Mvalue,RFM_score
0,259266,2656,1,320608.80,1,1,5,115
1,259287,2333,7,17565.78,5,5,5,555
2,259308,2406,4,6954.48,2,4,4,244
3,259329,2349,1,7211.94,4,1,4,414
4,259350,2641,1,1387.20,1,1,2,112


In [16]:
rfmdf["RFM_score"] = rfmdf["RFM_score"].astype(int)

In [17]:
rfmdf = rfmdf.merge(rfm[["RFM_score", "Segments"]], on="RFM_score", how="left")

In [18]:
rfmdf.to_csv("../dataset/rfmdf.csv", index=False)

Done!